In [ ]:
#| default_exp format
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
import sys, os
from pathlib import Path

In [ ]:
#| hide

# Insert in Path Project Directory
sys.path.insert(0, str(Path.cwd().parent))

In [ ]:
# | export
import re
from typing import List, Tuple, Union

import pandas as pd
from dotenv import find_dotenv, load_dotenv
from fastcore.utils import listify
from fastcore.xtras import Path
from geopy.distance import geodesic
from pyarrow import ArrowInvalid

from extracao.constants import APP_ANALISE_EN, APP_ANALISE_PT, BW, RE_BW

MAX_DIST = 10  # Km
LIMIT_FREQ = 84812.50
load_dotenv(find_dotenv(), override=True)

True

In [ ]:
def _read_df(folder: Union[str, Path], stem: str) -> pd.DataFrame:
    """Lê o dataframe formado por folder / stem.[parquet.gzip | fth | xslx]"""
    file = Path(f"{folder}/{stem}.parquet.gzip")
    try:
        df = pd.read_parquet(file)
    except (ArrowInvalid, FileNotFoundError) as e:
        raise e(f"Error when reading {file}") from e
    return df

# Formatação

> Este módulo possui funções auxiliares de formatação dos dados das várias fontes.


In [ ]:
# | export
def parse_bw(
    bw: str,  # Designação de Emissão (Largura + Classe) codificada como string
) -> Tuple[str, str]:  # Largura e Classe de Emissão
    """Parse the bandwidth string"""
    if match := re.match(RE_BW, bw):
        multiplier = BW[match[2]]
        if mantissa := match[3]:
            number = float(f"{match[1]}.{mantissa}")
        else:
            number = float(match[1])
        classe = match[4]
        return str(multiplier * number), str(classe)
    return pd.NA, pd.NA

In [ ]:
# | export
def _filter_matlab(
    df: pd.DataFrame,  # Arquivo de Dados Base de Entrada
) -> pd.DataFrame:  # Arquivo de Dados formatado para leitura no Matlab
    """Recebe a base de dados da Anatel e formata as colunas para leitura de acordo com os requisitos do Matlab"""
    df["#Estação"] = df["Estação"]
    df.loc[df.Multiplicidade != "1", "#Estação"] = (
        df.loc[df.Multiplicidade != "1", "Estação"]
        + "+"
        + df.loc[df.Multiplicidade != "1", "Multiplicidade"]
    )
    cols_desc = [
        "Fonte",
        "Status",
        "Classe",
        "Entidade",
        "Fistel",
        "#Estação",
        "Município_IBGE",
        "UF",
    ]
    df.loc[:, cols_desc] = df.loc[:, cols_desc].astype("string").fillna("NI")

    df["Descrição"] = (
        "["
        + df.Fonte
        + "] "
        + df.Status
        + ", "
        + df.Classe
        + ", "
        + df.Entidade.str.title()
        + " ("
        + df.Fistel
        + ", "
        + df["#Estação"]
        + "), "
        + df.Município_IBGE
        + "/"
        + df.UF
    )

    bad_coords = df.Coords_Valida_IBGE == "0"

    df.loc[bad_coords, "Descrição"] = df.loc[bad_coords, "Descrição"] + "*"

    df.loc[bad_coords, ["Latitude", "Longitude"]] = df.loc[
        bad_coords, ["Latitude_IBGE", "Longitude_IBGE"]
    ].values

    df = df.loc[:, APP_ANALISE_PT]
    df.columns = APP_ANALISE_EN
    return df

In [ ]:
# | export
def _format_matlab(
    df: pd.DataFrame,  # Arquivo de Dados Base de Entrada
) -> pd.DataFrame:  # Arquivo de Dados formatado para leitura no Matlab
    """Formata o arquivo final de dados para o formato esperado pela aplicação em Matlab"""
    df = df.astype("string")
    df.loc[len(df), :] = [
        "-1",
        "-15.7801",
        "-47.9292",
        "[TEMP] L, FX, Estação do SMP licenciada (cadastro temporário)",
        "10",
        "999999999",
        "NI",
        "-1",
    ]  # Paliativo...
    for c in df.columns:
        if c not in ["Description", "Class"]:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    for c in ["Latitude", "Longitude"]:
        df[c] = df[c].fillna(-1).astype("float32")
    df["Frequency"] = df["Frequency"].astype("float64")
    df["Service"] = df.Service.fillna(-1).astype("int16")
    df["Station"] = df.Station.fillna(-1).astype("int32")
    df["BW"] = df["BW"].fillna(-1).astype("float32")
    df.loc[df["Class"].isin(["", "-1"]), "Class"] = pd.NA
    df["Class"] = df.Class.fillna("NI").astype("category")
    df = df[df.Frequency <= LIMIT_FREQ]
    df.sort_values(
        by=["Frequency", "Latitude", "Longitude", "Description"], inplace=True
    )
    unique_columns = df.columns.tolist()
    unique_columns.remove("Description")
    df = df.drop_duplicates(subset=unique_columns, keep="last").reset_index(drop=True)
    df["Id"] = [f"#{i+1}" for i in df.index]
    df["Id"] = df.Id.astype("string")
    df.loc[df.Description == "", "Description"] = pd.NA
    df["Description"] = df["Description"].astype("string").fillna("NI")
    return df[["Id"] + list(APP_ANALISE_EN)]

### Mesclagem
Função auxiliar para mesclar registros que são iguais das diversas bases, i.e. estão a uma distância menor que `MAX_DIST` e verificar a validade da mesclagem

In [ ]:
# | export
def get_km_distance(row):
    return geodesic((row.iloc[0], row.iloc[1]), (row.iloc[2], row.iloc[3])).km


def merge_on_frequency(
    df_left: pd.DataFrame,  # DataFrame da esquerda a ser mesclado
    df_right: pd.DataFrame,  # DataFrame da direira a ser mesclado
    on: str = "Frequência",  # Coluna usada como chave de mesclagem
    cols2merge: Tuple = ("Entidade", "Fonte"),  # Colunas a serem mescladas
) -> pd.DataFrame:  # DataFrame resultante da mesclagem
    """Mescla os dataframes baseados na frequência
    É assumido que as colunas de ambos uma é subconjunto ou idêntica à outra, caso contrário os filtros não irão funcionar como esperado
    """
    df_left["Frequência"] = df_left["Frequência"].astype(
        "string[pyarrow]"
    )  # .drop_duplicates(ignore_index=True)
    df_right["Frequência"] = df_right["Frequência"].astype(
        "string[pyarrow]"
    )  # .drop_duplicates(ignore_index=True)
    df: pd.DataFrame = pd.merge(
        df_left,
        df_right,
        on=on,
        how="outer",
        indicator=True,
        copy=False,
    )

    left_suffix, right_suffix = "_x", "_y"
    lat, long = "Latitude", "Longitude"

    left_only = df._merge == "left_only"
    right_only = df._merge == "right_only"
    both = df._merge == "both"
    df = df.drop(columns=["_merge"])

    # Disjuntos
    if df[both].empty:
        return pd.concat([df_left, df_right], ignore_index=True)

    left_cols = [c for c in df.columns if right_suffix not in c]
    right_cols = [c for c in df.columns if left_suffix not in c]

    only_left = (
        df.loc[left_only, left_cols]
        .copy()
        .drop_duplicates(subset=left_cols, ignore_index=True)
    )
    only_left.columns = [c.replace(left_suffix, "") for c in left_cols]

    only_right = (
        df.loc[right_only, right_cols]
        .copy()
        .drop_duplicates(subset=right_cols, ignore_index=True)
    )
    only_right.columns = [c.replace(right_suffix, "") for c in right_cols]

    intersection_left = len(df_left) - len(only_left)
    intersection_right = len(df_right) - len(only_right)

    both_columns = [
        f"{lat}{left_suffix}",
        f"{long}{left_suffix}",
        f"{lat}{right_suffix}",
        f"{long}{right_suffix}",
    ]
    df.loc[both, "Distance"] = df.loc[both, both_columns].apply(get_km_distance, axis=1)

    df_both = df[both].sort_values("Distance", ignore_index=True)

    filter_left_cols = df_both.columns[: len(df_left.columns)].to_list()
    filter_right_cols = (
        listify(on) + df_both.columns[len(df_left.columns) : -1].to_list()
    )  # the -1 is to eliminate the distance

    # keep only the closer merged rows in the outer join
    df_both_left = df_both.copy().drop_duplicates(
        filter_left_cols, keep="first", ignore_index=True
    )
    df_both_right = df_both.copy().drop_duplicates(
        filter_right_cols, keep="first", ignore_index=True
    )

    # Sanity Checks
    assert (
        len(df_both_left) == intersection_left
    ), f"Grouping by unique columns has unexpected length: {len(df_both_left)}!= {intersection_left}"

    assert (
        len(df_both_right) == intersection_right
    ), f"Grouping by unique columns has unexpected length: {len(df_both_right)}!= {intersection_right}"

    # Separate according the MAX_DIST
    df_both_far_left = df_both_left[df_both_left.Distance > MAX_DIST]
    df_both_far_right = df_both_right[df_both_right.Distance > MAX_DIST]

    df_both_left = df_both_left[df_both_left.Distance <= MAX_DIST]
    df_both_right = df_both_right[df_both_right.Distance <= MAX_DIST]

    merge_cols = df_both.columns.to_list()
    merge_cols.remove("Distance")
    # Since it's an outer join, keep only the columns that are in both
    df_close_merge = (
        pd.merge(df_both_left, df_both_right, how="inner", on=merge_cols, copy=False)
        .drop("Distance_y", axis=1)
        .rename(columns={"Distance_x": "Distance"})
    )

    df_both_left = _left_filter(df_both_left, df_close_merge, merge_cols)
    df_both_right = _left_filter(df_both_right, df_close_merge, merge_cols)

    assert pd.merge(
        df_both_left, df_both_right, how="inner", on=merge_cols
    ).empty, "Check merging steps, df_both_left and df_both_right should be disjoint"

    df_final_merge = pd.concat(
        [df_close_merge, df_both_left, df_both_right], ignore_index=True
    )

    assert len(df_both_far_left) + len(df_close_merge) + len(df_both_left) == (
        len(df_left) - len(only_left)
    ), "Check merging steps, validation failed!"

    assert len(df_both_far_right) + len(df_close_merge) + len(df_both_right) == (
        len(df_right) - len(only_right)
    ), "Check merging steps, validation failed!"

    original_cols = df_both.columns.to_list()
    df_both = (
        pd.merge(
            df_both, df_final_merge, how="left", on=filter_left_cols, indicator=True
        )
        .loc[lambda x: x["_merge"] == "left_only"]
        .iloc[:, range(len(original_cols))]
    )
    df_both.columns = original_cols

    df_both = (
        pd.merge(
            df_both, df_final_merge, how="left", on=filter_right_cols, indicator=True
        )
        .loc[lambda x: x["_merge"] == "left_only"]
        .iloc[:, range(len(original_cols))]
    )
    df_both.columns = original_cols

    assert (
        df_both.Distance > MAX_DIST
    ).all(), "Check merging steps, validation failed!"

    assert (
        pd.merge(
            df_both,
            df_both_far_left,
            how="left",
            on=filter_left_cols,
            indicator=True,
            copy=False,
        )
        .loc[lambda x: x["_merge"] == "left_only"]
        .iloc[:, range(len(original_cols))]
        .empty
    ), "Check merging steps, validation failed!"

    assert (
        pd.merge(
            df_both,
            df_both_far_right,
            how="left",
            on=filter_right_cols,
            indicator=True,
            copy=False,
        )
        .loc[lambda x: x["_merge"] == "left_only"]
        .iloc[:, range(len(original_cols))]
        .empty
    ), "Check merging steps, validation failed!"

    for col in cols2merge:
        df_final_merge[f"{col}{left_suffix}"] = (
            df_final_merge[f"{col}{left_suffix}"]
            + " | "
            + df_final_merge[f"{col}{right_suffix}"]
        )

    df_final_merge = df_final_merge[left_cols]
    df_final_merge.columns = only_left.columns

    df_both_far_left = df_both_far_left[left_cols]
    df_both_far_left.columns = only_left.columns

    df_both_far_right = df_both_far_right[right_cols]
    df_both_far_right.columns = only_right.columns

    return pd.concat(
        [only_left, df_both_far_left, df_final_merge, only_right, df_both_far_right],
        ignore_index=True,
    )


def _left_filter(df, df_close_merge, merge_cols):
    df = pd.merge(
        df, df_close_merge, how="left", on=merge_cols, indicator=True, copy=False
    )
    df = df.loc[df["_merge"] == "left_only"]
    return df.drop(["_merge", "Distance_y"], axis=1).rename(
        columns={"Distance_x": "Distance"}
    )

In [ ]:
#| eval: false
folder = Path.cwd().parent / "dados"
